# Часть IV. Байесовская классификация #


### Задание: ###

 - 4.1. Построить байесовский классификатор в предположении, что переменная распределена нормально для обоих классов (5 баллов).
 - 4.2. Оценить точность классификации при случайном разбиении выборки на обучающую (80%) и контрольную выборки (20%) (5 баллов).
 
 
### Данные (для варианта № 10): ###

 - Классы: Iris Setosa, Iris versicolor
 - Переменная: Ширина чашелистика
 
 
#### Выполнила: #### 
*Нурдолотова Сабина, группа 20930* 


#### Заключение: #### 
- Был построен Байесовский классификатор, основанный на принципе максимума апостериорной вероятности. Для классифицируемого объекта были вычислены функции правдоподобия каждого из классов, по ним - апостериорные вероятности классов. Так объект относится к тому классу, для которого апостериорная вероятность максимальна.



- Была вычислена точность классификации при случайном разбиении выборки на обучающую (80%) и контрольную выборки (20%):
  
  Средняя доля правильных ответов составляет : 85.0

In [2]:
# импорт библиотек
import numpy as np
import pandas as pd
from sklearn import datasets
from pandas import DataFrame
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# импорт данных

iris = pd.read_csv('iris.csv')
iris = pd.DataFrame(iris, columns=['SepalWidth', 'Species'])
iris = iris[(iris['Species'] != 'Iris-virginica')]

# Эта функция разделяет набор данных на обучающую (80%) и контрольную выборки (20%)
def split_train_test(iris):
    X = iris.iloc[:,:1].values
    y = iris['Species'].values
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.20, random_state=42)
    return  X_train, X_test, Y_train, Y_test

# Подготовка целевых данных - кодируем значения классов: Iris-setosa = 0, Iris-versicolor = 1
def target( X_train, X_test, Y_train, Y_test ):


    for i in range(len(Y_train)):
        if Y_train[i] == 'Iris-setosa':
            Y_train[i] = 0
        if Y_train[i] ==  'Iris-versicolor' :
            Y_train[i] = 1


    for i in range(len(Y_test)):
        if Y_test[i] == 'Iris-setosa':
            Y_test[i] = 0
        if Y_test[i] ==  'Iris-versicolor' :
            Y_test[i] = 1

    
    return X_train, X_test, Y_train, Y_test

class NaiveBayes:
    
    def fit(self, X, y):
        # n_samples = 80 и n_features = 1
        n_samples, n_features = len(X), len(X[0])
        self._classes = np.unique(y)
        n_classes = len(self._classes)
        
        # инициализируем mean, var, priors
        self._mean = np.zeros((n_classes, n_features), dtype = np.float64)
        self._var = np.zeros((n_classes, n_features), dtype = np.float64)
        self._priors = np.zeros(n_classes, dtype = np.float64)
        
        # разделение на классы
        C = np.array([X[:40],X[40:]])
        
        # вычисление средних значений, отклонений и априорных вероятностей для всех классов
        for c in self._classes :
            X_c = C[c]
            self._mean[c] = np.mean(X_c, axis=0)
            self._var[c] = np.var(X_c, axis=0)
            self._priors[c] = X_c.shape[0] / float(n_samples)

    # эта функция создает список предсказаний для тестовых данных
    def predict(self, X):
        y_pred = [self._predict(x) for x in X]
        return y_pred

    # эта функция выполняет прогнозирование для выборочного семпла тестовых данных
    def _predict(self, x):
        posteriors = []

        for i, c in enumerate(self._classes):

            # Log is a increasing function we are using it for optimizing,
            # ,computations of posteriors probabilities to reduce overflow problem
            prior = np.log(self._priors[i])
            class_conditional = np.sum(np.log(self._pdf(i, x)))
            posterior = class_conditional + prior
            posteriors.append(posterior)

        return self._classes[np.argmax(posteriors)]

    # с предположением о независимости 
    # вычисляем pdf для всех условных вероятностей с учетом нормального распределения 
    def _pdf(self, class_i, x):
        mean = self._mean[class_i]
        var = self._var[class_i]
        numerator = np.exp( -(x - mean)**2 / (2*var) )
        denominator = np.sqrt(2 * np.pi * var)
        return numerator / denominator
        
    # эта функция вычисляет точность байесовских предсказаний
    def accuracy(self, y_true, y_pred):
        n = len(y_true)
        match = []
        for i in range(n):
            match.append(y_true[i]== y_pred[i])
        acc = np.sum(match) / n
        return acc
    
    
    
# функция NB_10_times повторяет NaiveBayes 10 раз
# Выводит среднюю точность 10 итераций

def NB_10_times(iris):
    accuracies = []       # точность 10 итераций    
    for j in range(10):
        # разделение набора данных
        X_train, X_test, Y_train, Y_test  = split_train_test(iris)
 
        # метки
        X_train, X_test, Y_train, Y_test = target( X_train, X_test, Y_train, Y_test)
        
       
        nb = NaiveBayes()
        nb.fit(X_train, Y_train)
        predictions = nb.predict(X_test)

        # acc_i is the accuracy of the current iteration
        acc_i = nb.accuracy(Y_test, predictions)
        accuracies.append(acc_i*100)

        # Expected result vs predicted
        #dic = {'Expected' : Y_test, 'Predicted' : predictions}
        #res = pd.DataFrame(dic)
        #print(res)
        #print(predictions)
        #print("\n Iteration",j+1,"\nThe Expected : ",Y_test)
        #print("The Predicted : ",predictions)
        #print("\nNaiveBayes classification accuracy is : ",acc_i*100)
        #print()
    
    avg_acc = np.mean(accuracies)
 
    return avg_acc



avg_accuracy = NB_10_times(iris)

print(f"\nСредняя доля правильных ответов составляет :",avg_accuracy)



Средняя доля правильных ответов составляет : 85.0
